# IBM Capstone Project
This file will hold my code for the IBM Data Science Capstone Project

## Introduction
As part of the Coursera Data Science Capstone project, I will be using Python and Coursera to analyse the number of Italian restaurants by postcode in the Greater London area.   This is to aid in the understanding of the relative density of restaurants to then inform entrepreneurs as to the area with the greatest opportunity for new restaurants.

London has one of the greatest plethora of restaurants in the world, with hundreds of new restaurants opening every month in the capital, however being a highly competitive market, there are also hundreds of restaurants that close every month.  Understanding the "right" location to launch a new restaurant is an important decision for any restaurateur.  This piece of analysis will be focused on Italian restaurants, to limit the immediate scope and to test the functionality of the tool, however the analysis could be broadened at a late time to cater to a users desired input cuisine.

## Data
Throughout this analysis I will be using Python and a range of libraries which have been pre-install or added.  The below code inputs the required tools:

In [4]:
# Import pandas and numpy
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('numpy, pandas, ..., imported...')

# library for BeautifulSoup, for web scrapping
from bs4 import BeautifulSoup

# library to handle JSON files
import json

!pip -q install geopy
print('geopy installed...')

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
print('Nominatim imported...')

# library to handle requests
import requests
print('requests imported...')

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
print('json_normalize imported...')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('matplotlib imported...')

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Kmeans imported...')

# install the Geocoder
!pip -q install geocoder
import geocoder

!pip -q install folium
print('folium installed...')
import folium # map rendering library
print('folium imported...')
print('...Done')

numpy, pandas, ..., imported...
geopy installed...
Nominatim imported...
requests imported...
json_normalize imported...
matplotlib imported...
Kmeans imported...
folium installed...
folium imported...
...Done


#### Location data
This analysis will require data to be accessed from Foursquare, a location data provider.  This includes information about the number, location, ratings and many more details regarding venues across the world.  I will use this information to extract my data, for which I will be focusing on the data associated with London and Italian restaurants (API ID: 4bf58dd8d48988d110941735).  This will enable me to gauge the number of Italian restaurants in the city but then associate it with given postcodes.

#### London data
To filter the data from Foursquare for London, I will need to identify the postcode areas associated with London.  I will use Wikipedia for this.  The attached [webpage](https://en.wikipedia.org/wiki/List_of_areas_of_London) is where I have pulled the information.  However this will need cleansing, which is why I installed BeautifulSoup above (a HTML parsing and editing library).